# O PROBLEMA

Imagine agora que você foi contratado(a) como Expert em Data Analytics por um grande hospital para entender como foi o comportamento da população na época da pandemia da COVID-19 e quais indicadores seriam importantes para o planejamento, caso haja um novo surto da doença.

Apesar de ser contratado(a) agora, a sua área observou que a utilização do estudo do PNAD-COVID 19 do IBGE seria uma ótima base para termos boas respostas ao problema proposto, pois são dados confiáveis. Porém, não será necessário utilizar todas as perguntas realizadas na pesquisa para enxergar todas as oportunidades ali postas.

É sempre bom ressaltar que há dados triviais que precisam estar no projeto, pois auxiliam muito na análise dos dados:

## PNAD-COVID-19 do IBGE

O Head de Dados pediu para que você entrasse na base de dados do PNAD-COVID-19 do IBGE e organizasse esta base para análise, utilizando Banco de Dados em Nuvem e trazendo as seguintes características:

- a. Utilização de no máximo 20 questionamentos realizados na pesquisa;
- b. Utilizar 3 meses para construção da solução;
- c. Caracterização dos sintomas clínicos da população;
- d. Comportamento da população na época da COVID-19;
- e. Características econômicas da Sociedade;

Seu objetivo será trazer uma breve análise dessas informações, como foi a organização do banco, as perguntas selecionadas para a resposta do problema e quais seriam as principais ações que o hospital deverá tomar em caso de um novo surto de COVID-19.

- Dica: Leiam com atenção a base de dados e toda a documentação que o site o PNAD – Covid19 traz, principalmente os dicionários, que ajudam e muito no entendimento da Base de Dados.
- Dica 2: Utilizem o que já foi ensinado e consolidado nas outras fases para apresentar a resolução do projeto.

Lembre-se de que você poderá apresentar o desenvolvimento do seu projeto durante as lives com docentes. Essa é uma boa oportunidade para discutir sobre as dificuldades encontradas e pegar dicas valiosas com especialistas e colegas de turma.

Link para a base: https://www.ibge.gov.br/estatisticas/investigacoes-experimentais/estatisticas-experimentais/27946-divulgacao-semanal-pnadcovid1?t=downloads&utm_source=covid19&utm_medium=hotsite&utm_campaign=covid_19

Outras fontes:

https://www.gov.br/saude/pt-br/assuntos/noticias/2021/janeiro/entenda-a-ordem-de-vacinacao-contra-a-covid-19-entre-os-grupos-prioritarios

https://www.gov.br/saude/pt-br/assuntos/coronavirus/vacinas/esquemas-vacinais

### Importando bibliotecas

In [1]:
import warnings 
warnings.filterwarnings(action = 'ignore')

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.sql import text
from pyspark.sql import SparkSession
import findspark
import os
import pandas as pd
import polars as pl
import plotly.express as px
import plotly.graph_objects as go

### Conectando ao MySQL

In [3]:
'''def mysql_connection(host, user, passwd, database=None):
    engine = create_engine(f'mysql+pymysql://{user}:{passwd}@{host}/{database}')
    return engine.connect()

mysql = mysql_connection('127.0.0.1', 'root', 'admin', 'pnad_covid')'''

"def mysql_connection(host, user, passwd, database=None):\n    engine = create_engine(f'mysql+pymysql://{user}:{passwd}@{host}/{database}')\n    return engine.connect()\n\nmysql = mysql_connection('127.0.0.1', 'root', 'admin', 'pnad_covid')"

In [4]:
'''statement = text('SELECT * FROM pnad_covid_view LIMIT 5')
response = mysql.execute(statement)
for row in response:
        print(row)'''

"statement = text('SELECT * FROM pnad_covid_view LIMIT 5')\nresponse = mysql.execute(statement)\nfor row in response:\n        print(row)"

### Criando sessão Spark

In [5]:
os.environ['SPARK_HOME'] = '/opt/spark/'
findspark.init()

In [6]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

23/10/27 17:01:08 WARN Utils: Your hostname, platero-Lenovo-IdeaPad-S145-15IWL resolves to a loopback address: 127.0.1.1; using 192.168.15.109 instead (on interface wlp2s0)
23/10/27 17:01:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/27 17:01:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
spark

### Importando dados

In [8]:
'''res = mysql.execute(text('SELECT * FROM pnad_covid_view LIMIT 1149196'))
res = res.fetchall()'''

"res = mysql.execute(text('SELECT * FROM pnad_covid_view LIMIT 1149196'))\nres = res.fetchall()"

In [9]:
'''df = pd.DataFrame(res)'''

'df = pd.DataFrame(res)'

In [10]:
'''df.to_csv('dados/dados_exportados/2023-10-27_pnad_covid_view.csv', index=False)'''

"df.to_csv('dados/dados_exportados/2023-10-27_pnad_covid_view.csv', index=False)"

### Dicionários

In [11]:
estados = {
    'Acre': 'AC',
    'Alagoas': 'AL',
    'Amapá': 'AP',
    'Amazonas': 'AM',
    'Bahia': 'BA',
    'Ceará': 'CE', 
    'Distrito Federal': 'DF',
    'Espírito Santo': 'ES',
    'Goiás': 'GO',
    'Maranhão': 'MA',
    'Mato Grosso': 'MT',
    'Mato Grosso do Sul': 'MS',
    'Minas Gerais': 'MG',
    'Pará': 'PA',
    'Paraíba': 'PB',
    'Paraná': 'PR',
    'Pernambuco': 'PE',
    'Piauí': 'PI', 
    'Rio de Janeiro': 'RJ',
    'Rio Grande do Norte': 'RN',
    'Rio Grande do Sul': 'RS',
    'Rondônia': 'RO',
    'Roraima': 'RR',
    'Santa Catarina': 'SC',
    'São Paulo': 'SP',
    'Sergipe': 'SE',
    'Tocantins': 'TO'
}

regioes = {
    'AC': 'Norte',
    'AL': 'Nordeste',
    'AP': 'Norte',
    'AM': 'Norte',
    'BA': 'Nordeste',
    'CE': 'Nordeste',
    'DF': 'Centro-Oeste',
    'ES': 'Sudeste',
    'GO': 'Centro-Oeste',
    'MA': 'Nordeste',
    'MT': 'Centro-Oeste',
    'MS': 'Centro-Oeste', 
    'MG': 'Sudeste',
    'PA': 'Norte',
    'PB': 'Nordeste',
    'PR': 'Sul',
    'PE': 'Nordeste',
    'PI': 'Nordeste',
    'RJ': 'Sudeste',
    'RN': 'Nordeste',
    'RS': 'Sul',
    'RO': 'Norte',
    'RR': 'Norte',
    'SC': 'Sul',
    'SP': 'Sudeste',
    'SE': 'Nordeste',
    'TO': 'Norte'
}

### Análise exploratória de dados econômicos

In [12]:
df = spark.read.csv('dados/dados_exportados/2023-10-27_pnad_covid_view.csv', sep=',', inferSchema=True, header=True)

In [13]:
df.show(5, truncate=False)

23/10/27 17:01:28 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+--------------+------------------+-----+------+--------+------------------------------------+-----------------------------+-----------------------+-----------------------------+---------------------------------+--------------------+------------------------+------------------+---------------------------------+--------------------------+---------------------------+-------------------------------+----------------+-------------------------------+------------------------+---------------------+-----------------------+-----------------+-----------------------+--------------------+-------------+-----------------------+-----------+---------------------------+-------------+-----------------+---------------+
|data      |uf            |situacao_domicilio|idade|sexo  |cor_raca|escolaridade                        |questao_estabelecimento_saude|questao_permaneceu_casa|questao_remedio_conta_propria|questao_remedio_orientacao_medica|questao_hospital_SUS|questao_hospital_privado|questao_inte

In [14]:
len(df.columns)

33

In [15]:
df.columns

['data',
 'uf',
 'situacao_domicilio',
 'idade',
 'sexo',
 'cor_raca',
 'escolaridade',
 'questao_estabelecimento_saude',
 'questao_permaneceu_casa',
 'questao_remedio_conta_propria',
 'questao_remedio_orientacao_medica',
 'questao_hospital_SUS',
 'questao_hospital_privado',
 'questao_internacao',
 'questao_internacao_ajuda_respirar',
 'questao_motivo_afastamento',
 'questao_tempo_afastado_trab',
 'questao_tipo_trabalho_realizado',
 'faixa_rendimento',
 'rendimento_aposentadoria_pensao',
 'rendimento_bolsa_familia',
 'rendimento_beneficios',
 'auxlio_emergencia_covid',
 'seguro_desemprego',
 'tipo_domicilio',
 'valor_pago_domicilio',
 'sintoma_covid',
 'descricao_sintoma_covid',
 'teste_covid',
 'descricao_fator_risco_covid',
 'tipo_teste',
 'fator_risco_covid',
 'resultado_teste']

In [16]:
df.printSchema()

root
 |-- data: date (nullable = true)
 |-- uf: string (nullable = true)
 |-- situacao_domicilio: string (nullable = true)
 |-- idade: integer (nullable = true)
 |-- sexo: string (nullable = true)
 |-- cor_raca: string (nullable = true)
 |-- escolaridade: string (nullable = true)
 |-- questao_estabelecimento_saude: string (nullable = true)
 |-- questao_permaneceu_casa: string (nullable = true)
 |-- questao_remedio_conta_propria: string (nullable = true)
 |-- questao_remedio_orientacao_medica: string (nullable = true)
 |-- questao_hospital_SUS: string (nullable = true)
 |-- questao_hospital_privado: string (nullable = true)
 |-- questao_internacao: string (nullable = true)
 |-- questao_internacao_ajuda_respirar: string (nullable = true)
 |-- questao_motivo_afastamento: string (nullable = true)
 |-- questao_tempo_afastado_trab: string (nullable = true)
 |-- questao_tipo_trabalho_realizado: string (nullable = true)
 |-- faixa_rendimento: string (nullable = true)
 |-- rendimento_aposentado

#### Adicionando colunas

In [17]:
df = df.withColumn('sigla', df.uf).replace(to_replace=estados, subset=['sigla']) #adicionando sigla dos estados
df = df.withColumn('regiao', df.sigla).replace(to_replace=regioes, subset=['regiao']) #adicionando regioes dos estados
df.show(5, truncate=False)

+----------+--------------+------------------+-----+------+--------+------------------------------------+-----------------------------+-----------------------+-----------------------------+---------------------------------+--------------------+------------------------+------------------+---------------------------------+--------------------------+---------------------------+-------------------------------+----------------+-------------------------------+------------------------+---------------------+-----------------------+-----------------+-----------------------+--------------------+-------------+-----------------------+-----------+---------------------------+-------------+-----------------+---------------+-----+--------+
|data      |uf            |situacao_domicilio|idade|sexo  |cor_raca|escolaridade                        |questao_estabelecimento_saude|questao_permaneceu_casa|questao_remedio_conta_propria|questao_remedio_orientacao_medica|questao_hospital_SUS|questao_hospital_priva

In [18]:
df_temp = df.createOrReplaceTempView('df_temp') #criando tabela temporária na sessão Spark SQL

In [19]:
spark.sql(
    '''
        SELECT faixa_rendimento
        FROM df_temp 
        WHERE faixa_rendimento != 'Não aplicável' AND faixa_rendimento != '101 - 300'
        LIMIT 10
    '''
).toPandas()

faixa_rendimento
0      801 - 1.600
1      801 - 1.600
2      801 - 1.600
3      801 - 1.600
4      801 - 1.600
5    1.601 - 3.000
6        301 - 600
7   3.001 - 10.000
8  10.001 - 50.000
9      801 - 1.600

#### Qual é a porcentagem de infectados entre pacientes testados por zona?

In [20]:
df_testes_validos_zona = spark.sql(
    '''
        SELECT situacao_domicilio, count(resultado_teste) AS qtd_testes_validos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo' OR resultado_teste = 'Negativo' 
        GROUP BY situacao_domicilio
    '''
).toPandas()

df_positivos_zona = spark.sql(
    '''
        SELECT situacao_domicilio, count(resultado_teste) AS qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY situacao_domicilio
    '''
).toPandas()


df_infectados_zona = pd.merge(df_testes_validos_zona, df_positivos_zona, on='situacao_domicilio')
df_infectados_zona['percentual_infectados'] = ((df_infectados_zona.qtd_infectados / df_infectados_zona.qtd_testes_validos) * 100).round(decimals=2)

fig = px.bar(
    data_frame=df_infectados_zona.sort_values('percentual_infectados'),
    x='situacao_domicilio',
    y='percentual_infectados',
    color = 'situacao_domicilio'
)

fig.update_layout(
    title='Porcentagem de infectados entre pacientes testados por zona',
    xaxis_title='Zona', 
    yaxis_title='%',
    legend_title='Legenda',
    width=800, 
    height=600
)

fig.show()

#### Qual é porcentagem de infectados entre testados por gênero?

In [21]:
df_testes_validos_sexo = spark.sql(
    '''
        SELECT sexo, count(resultado_teste) AS qtd_testes_validos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo' OR resultado_teste = 'Negativo' 
        GROUP BY sexo
    '''
).toPandas()

df_testes_positivos_sexo = spark.sql(
    '''
        SELECT sexo, count(resultado_teste) AS qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY sexo
    '''
).toPandas()

df_infectados_zona = pd.merge(df_testes_validos_sexo, df_testes_positivos_sexo, on='sexo')
df_infectados_zona['percentual_infectados'] = ((df_infectados_zona.qtd_infectados / df_infectados_zona.qtd_testes_validos) * 100).round(decimals=2)

fig = px.bar(
    data_frame=df_infectados_zona.sort_values('percentual_infectados'),
    x='sexo',
    y='percentual_infectados',
    color='sexo'
)

fig.update_layout(
    title='Porcentagem de infectados entre pacientes testados por gênero',
    xaxis_title='Sexo', 
    yaxis_title='%',
    showlegend=False,
    width=800, 
    height=600
)

fig.show()

#### Qual é porcentagem de infectados entre pacientes testados por cor/raça?

In [22]:
df_testes_validos_cor_raca = spark.sql(
    '''
        SELECT cor_raca, count(resultado_teste) AS qtd_testes_validos
        FROM df_temp 
        WHERE (resultado_teste = 'Positivo' OR resultado_teste = 'Negativo') AND cor_raca != 'Ignorado'
        GROUP BY cor_raca

    '''
).toPandas()


df_testes_positivos_raca = spark.sql(
    '''
        SELECT cor_raca, count(resultado_teste) AS qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo' AND cor_raca != 'Ignorado'
        GROUP BY cor_raca
    '''
).toPandas()

df_infectados_cor_raca = pd.merge(df_testes_validos_cor_raca, df_testes_positivos_raca, on='cor_raca')
df_infectados_cor_raca['percentual_infectados'] = ((df_infectados_cor_raca.qtd_infectados / df_infectados_cor_raca.qtd_testes_validos) * 100).round(decimals=2)

fig = px.bar(
    data_frame=df_infectados_cor_raca.sort_values('percentual_infectados'),
    x='cor_raca',
    y='percentual_infectados',
    color='cor_raca'
)

fig.update_layout(
    title='Porcentagem de infectados entre pacientes testados por cor/raça',
    xaxis_title='Cor/Raça', 
    yaxis_title='%',
    showlegend=False,
    width=800, 
    height=600
)

fig.show()

#### Qual é a porcentagem de infectados entre pacientes testados por escolaridade?

In [23]:
df_testes_validos_escolaridade = spark.sql(
    '''
        SELECT escolaridade, count(resultado_teste) AS qtd_testes_validos
        FROM df_temp 
        WHERE resultado_teste = 'Positivo' OR resultado_teste = 'Negativo' 
        GROUP BY escolaridade
        ORDER BY escolaridade
    '''
).toPandas()

df_testes_positivos_escolaridade = spark.sql(
    '''
        SELECT escolaridade, count(resultado_teste) AS qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY escolaridade
        ORDER BY escolaridade
    '''
).toPandas()

df_infectados_escolaridade = pd.merge(df_testes_validos_escolaridade, df_testes_positivos_escolaridade, on='escolaridade')
df_infectados_escolaridade['percentual_infectados'] = ((df_infectados_escolaridade.qtd_infectados / df_infectados_escolaridade.qtd_testes_validos) * 100).round(decimals=2)


fig = px.bar(
    data_frame=df_infectados_escolaridade.sort_values('percentual_infectados'),
    x='escolaridade',
    y='percentual_infectados',
    color='escolaridade'
)

fig.update_layout(
    title='Porcentagem de infectados entre pacientes testados por escolaridade',
    xaxis_title='Escolaridade', 
    yaxis_title='%',
    showlegend=False,
    width=800,  
    height=600
)

fig.show()

#### Qual é a quantidade total de infectados por estado?

In [24]:
df_cotagio_estado = spark.sql(
    '''
        SELECT uf, count(resultado_teste) AS qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY uf
        ORDER BY uf
    '''
).toPandas()

fig = px.bar(
    data_frame=df_cotagio_estado.sort_values('qtd_infectados'),
    x='uf',
    y='qtd_infectados',
    color='uf'
)

fig.update_layout(
    title='Quantidade total de infectados por estado',
    xaxis_title='Estado', 
    yaxis_title='Quantidade de infectados',
    showlegend=False,
    width=800, 
    height=600
)

fig.show()

#### Qual é quantidade total de infectados por região?

In [25]:
df_cotagio_regiao = spark.sql(
    '''
        SELECT regiao, count(resultado_teste) as qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY regiao
        ORDER BY regiao
    '''
).toPandas()

fig = px.bar(
    data_frame=df_cotagio_regiao.sort_values('qtd_infectados'),
    x='regiao',
    y='qtd_infectados',
    color='regiao'
)

fig.update_layout(
    title='Quantidade total de infectados por região',
    xaxis_title='Região', 
    yaxis_title='Quantidade de infectados',
    showlegend=False,
    width=800, 
    height=600
)

fig.show()

#### QUal é a taxa de incidência por estado no período?

In [26]:
df_qtd_testes_validos_estado = spark.sql(
    '''
        SELECT uf, count(teste_covid) AS qtd_testes_validos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND (resultado_teste = 'Positivo' OR resultado_teste = 'Negativo')
        GROUP BY uf
    '''
).toPandas()

df_qtd_testes_positivos_estado = spark.sql(
    '''
        SELECT uf, count(resultado_teste) AS qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY uf
    '''
).toPandas()

df_taxa_de_contagio_estado = pd.merge(df_qtd_testes_validos_estado, df_qtd_testes_positivos_estado, on='uf')
df_taxa_de_contagio_estado['taxa_de_contagio_mil_habitantes'] = ((df_taxa_de_contagio_estado['qtd_infectados'] / df_taxa_de_contagio_estado['qtd_testes_validos']) * 1000).round().astype(int)

fig = px.bar(
    data_frame=df_taxa_de_contagio_estado.sort_values('taxa_de_contagio_mil_habitantes'),
    x='uf',
    y='taxa_de_contagio_mil_habitantes',
    color='uf'
)

fig.update_layout(
    title='Taxa de incidência por estado nos últimos 3 meses',
    xaxis_title='Estado', 
    yaxis_title='Taxa de incidência por mil habitantes',
    showlegend=False,
    width=800, 
    height=600
)

fig.show()

#### Qual é a taxa de incidência por região no período?

In [27]:
df_qtd_testes_validos_regiao = spark.sql(
    '''
        SELECT regiao, count(teste_covid) AS qtd_testes_validos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND (resultado_teste = 'Positivo' OR resultado_teste = 'Negativo')
        GROUP BY regiao
    '''
).toPandas()


df_qtd_testes_positivos_regiao = spark.sql(
    '''
        SELECT regiao, count(resultado_teste) AS qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY regiao
    '''
).toPandas()

df_taxa_de_contagio_regiao = pd.merge(df_qtd_testes_validos_regiao, df_qtd_testes_positivos_regiao, on='regiao')
df_taxa_de_contagio_regiao['taxa_de_contagio_mil_habitantes'] = ((df_taxa_de_contagio_regiao['qtd_infectados'] / df_taxa_de_contagio_regiao['qtd_testes_validos']) * 1000).round().astype(int)

fig = px.bar(
    data_frame=df_taxa_de_contagio_regiao.sort_values('taxa_de_contagio_mil_habitantes'),
    x='regiao',
    y='taxa_de_contagio_mil_habitantes',
    color='regiao'
)

fig.update_layout(
    title='Taxa de incidência por região nos últimos 3 meses',
    xaxis_title='Região', 
    yaxis_title='Taxa de incidência por mil habitantes',
    showlegend=False,
    width=800, 
    height=600
)

fig.show()

#### Qual é a taxa de incidência por faixa etária do esquema vacinal nos estados?

In [28]:
df_qtd_testes_validos_faixa_etaria = spark.sql(
    '''
        SELECT
                (CASE
                        WHEN idade BETWEEN  1 AND 4 THEN '6 meses a 4 anos de idade'
                        WHEN idade BETWEEN 3 AND 4 THEN '3 e 4 anos de idade'
                        WHEN idade BETWEEN 5 AND 11 THEN '5 a 11 anos de idade'
                        WHEN idade BETWEEN 12 AND 39 THEN '12 a 39 anos de idade'
                        WHEN idade BETWEEN 40 AND 59 THEN '40 a 59 anos de idade'
                        WHEN idade >= 60 THEN 'Mais de 60 anos de idade'
                END) AS faixa_etaria,
                count(teste_covid) AS qtd_testes_validos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND (resultado_teste = 'Positivo' OR resultado_teste = 'Negativo')
        GROUP BY faixa_etaria
    '''
).toPandas()

df_qtd_testes_positivos_faixa_etaria = spark.sql(
    '''
        SELECT
                (CASE
                    WHEN idade BETWEEN  1 AND 4 THEN '6 meses a 4 anos de idade'
                    WHEN idade BETWEEN 3 AND 4 THEN '3 e 4 anos de idade'
                    WHEN idade BETWEEN 5 AND 11 THEN '5 a 11 anos de idade'
                    WHEN idade BETWEEN 12 AND 39 THEN '12 a 39 anos de idade'
                    WHEN idade BETWEEN 40 AND 59 THEN '40 a 59 anos de idade'
                    WHEN idade >= 60 THEN 'Mais de 60 anos de idade'
                END) AS faixa_etaria,
                count(resultado_teste) AS qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY faixa_etaria
    '''
).toPandas()

df_taxa_cotagio_faixa_etaria = pd.merge(df_qtd_testes_validos_faixa_etaria, df_qtd_testes_positivos_faixa_etaria, on='faixa_etaria')
df_taxa_cotagio_faixa_etaria['taxa_de_contagio_mil_habitantes']= ((df_taxa_cotagio_faixa_etaria['qtd_infectados'] / df_taxa_cotagio_faixa_etaria['qtd_testes_validos']) * 1000).round().astype(int)

fig = px.bar(
    data_frame=df_taxa_cotagio_faixa_etaria.sort_values('taxa_de_contagio_mil_habitantes'),
    x='faixa_etaria',
    y='taxa_de_contagio_mil_habitantes',
    color='faixa_etaria'
)

fig.update_layout(
    title='Taxa de incidência nos estados por faixa etária do esquema vacinal (por mil habitantes)',
    xaxis_title='Faixa etária', 
    yaxis_title='Taxa de incidência por mil habitantes',
    showlegend=False,
    width=800, 
    height=600
)

fig.show()

#### Qual é a quantidade de infectados por faixa de rendimento?

In [29]:
df_faixa_rendimento_infectados = spark.sql(
    '''
        SELECT faixa_rendimento, count(resultado_teste) AS qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY faixa_rendimento
    '''
).toPandas()

df_faixa_rendimento_infectados

faixa_rendimento  qtd_infectados
0         101 - 300             279
1   10.001 - 50.000             623
2              None           13728
3  50.001 - 100.000               7
4         301 - 600             661
5           0 - 100              45
6     1.601 - 3.000            5075
7       801 - 1.600            7645
8         601 - 800             399
9    3.001 - 10.000            3595

In [30]:
df_faixa_rendimento_infectados = spark.sql(
    '''
        SELECT faixa_rendimento, count(resultado_teste) AS qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY faixa_rendimento
    '''
).toPandas()

fig = px.bar(
    data_frame=df_faixa_rendimento_infectados.sort_values('qtd_infectados'),
    x='faixa_rendimento',
    y='qtd_infectados', 
    color='faixa_rendimento'
)

fig.update_layout(
    title='Quantidade de infectados por faixa de rendimento',
    xaxis_title='Faixa de rendimento (R$)', 
    yaxis_title='Quantidade',
    showlegend=False,
    width=800, 
    height=600
)

fig.show()

#### Qaal é quantidade total de testes por tipo de teste?

In [31]:
df_tipo_teste_covid = spark.sql(
    '''
        SELECT tipo_teste, count(resultado_teste) AS qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY tipo_teste
        ORDER BY tipo_teste
    '''
).toPandas()

fig = px.bar(
    data_frame=df_tipo_teste_covid.sort_values('qtd_infectados'),
    x='tipo_teste',
    y='qtd_infectados', 
    color='tipo_teste'
)

fig.update_layout(
    title='QUantidade total de testes por tipo de teste',
    xaxis_title='Tipo de teste', 
    yaxis_title='Quantidade',
    showlegend=False,
    width=800, 
    height=600
)

fig.show()

#### Que áreas profissionais foram mais afetadas pelo contagio no período?

##### Qual é a quantidade total de infectados por tipo de trabalho?

In [32]:
df_tipo_trabalho = spark.sql(
    '''
        SELECT questao_tipo_trabalho_realizado, count(resultado_teste) AS qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo' AND questao_tipo_trabalho_realizado != 'Não aplicável' AND questao_tipo_trabalho_realizado != 'Outros'
        GROUP BY questao_tipo_trabalho_realizado
        ORDER BY qtd_infectados DESC
    '''
).toPandas()

fig = px.bar(
    data_frame=df_tipo_trabalho,
    x='qtd_infectados',
    y='questao_tipo_trabalho_realizado', 
    color='questao_tipo_trabalho_realizado'
)

fig.update_layout(
    title='Quantidade total de infectados por tipo de trabalho',
    xaxis_title='Quantidade', 
    yaxis_title='Tipo de trabalho',
    showlegend=False,
    width=1200, 
    height=800
)

fig.show()

##### Qual é a porcentagem de infectados entre testados por tipo de trabalho?

In [33]:
df_qtd_testes_tipo_trabalho = spark.sql(
    '''
        SELECT questao_tipo_trabalho_realizado, count(teste_covid) AS qtd_testes_validos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND (resultado_teste = 'Positivo' OR resultado_teste = 'Negativo') AND questao_tipo_trabalho_realizado != 'Não aplicável' AND questao_tipo_trabalho_realizado != 'Outros'
        GROUP BY questao_tipo_trabalho_realizado
    '''
).toPandas()

df_qtd_testes_positivos_tipo_trabalho = spark.sql(
    '''
        SELECT questao_tipo_trabalho_realizado, count(resultado_teste) AS qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo' AND questao_tipo_trabalho_realizado != 'Não aplicável' AND questao_tipo_trabalho_realizado != 'Outros'
        GROUP BY questao_tipo_trabalho_realizado
    '''
).toPandas()

df_tipo_trabalho = pd.merge(df_qtd_testes_tipo_trabalho, df_qtd_testes_positivos_tipo_trabalho, on='questao_tipo_trabalho_realizado')
df_tipo_trabalho['percentual_infectados'] = ((df_tipo_trabalho['qtd_infectados'] / df_tipo_trabalho['qtd_testes_validos']) * 100).round().astype(int)

fig = px.bar(
    data_frame=df_tipo_trabalho.sort_values('percentual_infectados', ascending=False),
    x='percentual_infectados',
    y='questao_tipo_trabalho_realizado', 
    color='questao_tipo_trabalho_realizado'
)

fig.update_layout(
    title='Porcentagem de infectados entre testados por tipo de trabalho',
    xaxis_title='%', 
    yaxis_title='Tipo de trabalho',
    showlegend=False,
    width=1200, 
    height=800
)

fig.show()

In [39]:
import json

geojson = json.load(open('dados/dados_importados/brasil_estados.json'))

df_qtd_testes_validos_estado = spark.sql(
    '''
        SELECT sigla AS Sigla, count(teste_covid) AS qtd_testes_validos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND (resultado_teste = 'Positivo' OR resultado_teste = 'Negativo')
        GROUP BY Sigla
    '''
).toPandas()

df_qtd_testes_positivos_estado = spark.sql(
    '''
        SELECT sigla AS Sigla, count(resultado_teste) AS qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY Sigla
    '''
).toPandas()

df_taxa_de_contagio_estado = pd.merge(df_qtd_testes_validos_estado, df_qtd_testes_positivos_estado, on='Sigla')
df_taxa_de_contagio_estado['Taxa por mil habitantes'] = ((df_taxa_de_contagio_estado['qtd_infectados'] / df_taxa_de_contagio_estado['qtd_testes_validos']) * 1000).round().astype(int)


fig = px.choropleth(
    data_frame=df_taxa_de_contagio_estado,
    geojson=geojson,
    locations='Sigla',
    color='Taxa por mil habitantes',
    hover_data='Sigla',
    hover_name='Sigla',
    scope='south america',
    color_continuous_scale='reds',
)

fig.add_scattergeo(
    geojson=geojson,
    locations = df_taxa_de_contagio_estado['Sigla'],
    text = df_taxa_de_contagio_estado[f'Sigla'],
    textposition='middle center',
    mode = 'text',
    textfont=dict(
        size=14,
        color='black',
        
    )
)

fig.update_layout(
    title=dict(text='Taxa de incidência de Covid-19 por estado nos meses de setembro e outubro de 2021', font=dict(size=24)),
    width=800, 
    height=600
)

fig.show()

23/10/27 17:11:42 ERROR Executor: Exception in task 6.0 in stage 131.0 (TID 318)
org.apache.spark.SparkFileNotFoundException: File file:/home/platero/postech_techchallenge_fase_3/dados/dados_exportados/2023-10-27_pnad_covid_view.csv does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:780)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:220)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:279)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:129)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:4

Py4JJavaError: An error occurred while calling o287.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 131.0 failed 1 times, most recent failure: Lost task 4.0 in stage 131.0 (TID 316) (192.168.15.109 executor driver): org.apache.spark.SparkFileNotFoundException: File file:/home/platero/postech_techchallenge_fase_3/dados/dados_exportados/2023-10-27_pnad_covid_view.csv does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:780)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:220)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIter